In [13]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import sys
sys.path.append("..") #this is to add the above folder to the package directory
import os
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# from pandas.plotting import register_matplotlib_converters
import plotly.express as px
import matplotlib.pyplot as plt

In [15]:
%matplotlib inline 

## Increasing the temporal resolution 

## Step1: Reading the input file 

In [16]:
#reading all the sheets in the file:

folder_path = os.path.join('Data', 'electricity_data')

raw_CFdata = os.path.join(folder_path, 'CapacityFactor for Wind and Solar - All countries RN 2.xlsx')

sheetnames = ['AL Solar IN', 'AL Solar OUT', 'AL WPP IN', 'AL WPP OUT',
              'MK Solar IN', 'MK Solar OUT', 'MK WPP IN', 'MK WPP OUT',
              'ME Solar IN', 'ME Solar OUT', 'ME WPP IN', 'ME WPP OUT',
              'XS Solar IN', 'XS Solar OUT', 'XS WPP IN', 'XS WPP OUT']

list_of_dfs = [pd.read_excel(raw_CFdata, sheet_name=sheet, skiprows=3) for sheet in sheetnames]

In [17]:
filenames = sheetnames
for dataframe, filename in zip(list_of_dfs, filenames):
    dataframe['filename'] = filename

In [18]:
combined_df = pd.concat(list_of_dfs, ignore_index=True)

In [19]:
df = combined_df[['filename','local_time','electricity']].copy()


## Step2: Capacity Factor calculations 

In [20]:
df['year'] = df['local_time'].dt.year 
df['month'] = df['local_time'].dt.month
df['week'] = df['local_time'].dt.week
df['day'] = df['local_time'].dt.day 
df['hour'] = df['local_time'].dt.hour 

In [21]:
#avg = pd.pivot_table(df, values='electricity', index='hour',columns=['month'], aggfunc=np.mean, fill_value=0)
avg = pd.pivot_table(df, values='electricity', index='week', columns=['filename'], aggfunc=np.mean, fill_value=0)

In [24]:
all_cf = (avg/1000).round(3)  #since the installed capacity is 1000 KW
all_cf.reset_index(inplace=True)
all_cf.rename(columns={'week':'TS'}, inplace=True)
all_cf.drop([52], axis=0, inplace=True)

In [25]:
all_cf

filename,TS,AL Solar IN,AL Solar OUT,AL WPP IN,AL WPP OUT,ME Solar IN,ME Solar OUT,ME WPP IN,ME WPP OUT,MK Solar IN,MK Solar OUT,MK WPP IN,MK WPP OUT,XS Solar IN,XS Solar OUT,XS WPP IN,XS WPP OUT
0,1,0.046,0.152,0.0,0.266,0.167,0.063,0.392,0.362,0.046,0.111,0.260,0.312,0.048,0.066,0.242,0.288
1,2,0.071,0.157,0.0,0.243,0.124,0.065,0.293,0.224,0.058,0.109,0.176,0.216,0.059,0.059,0.091,0.165
2,3,0.111,0.151,0.0,0.210,0.151,0.114,0.117,0.208,0.072,0.158,0.084,0.196,0.086,0.130,0.106,0.232
3,4,0.085,0.065,0.0,0.264,0.068,0.078,0.311,0.256,0.072,0.063,0.145,0.244,0.086,0.092,0.132,0.238
4,5,0.063,0.070,0.0,0.508,0.111,0.053,0.349,0.431,0.086,0.066,0.270,0.245,0.062,0.066,0.186,0.343
5,6,0.059,0.094,0.0,0.164,0.061,0.067,0.356,0.224,0.082,0.076,0.226,0.269,0.052,0.068,0.169,0.191
6,7,0.165,0.192,0.0,0.420,0.233,0.117,0.494,0.478,0.208,0.194,0.219,0.244,0.101,0.120,0.338,0.378
7,8,0.210,0.184,0.0,0.214,0.216,0.183,0.317,0.313,0.210,0.195,0.186,0.213,0.146,0.199,0.235,0.261
8,9,0.117,0.091,0.0,0.174,0.102,0.088,0.260,0.287,0.130,0.127,0.094,0.120,0.137,0.116,0.196,0.278
9,10,0.086,0.143,0.0,0.221,0.162,0.067,0.090,0.289,0.115,0.100,0.347,0.348,0.076,0.104,0.184,0.264


## Step3: Generating and saving the CF tables for each technology and country

In [28]:
#The excel files will be saved in the same directory where this notebook exists

output_folder = os.path.join('Processed_data', 'Capacity_factors', 'Reference')
os.makedirs(output_folder, exist_ok = True)

names = all_cf.drop(['TS'], axis=1).columns

file_path=os.path.join(output_folder, 'Solar_and_Wind_CFs_20201029.xlsx')
writer = pd.ExcelWriter(file_path, engine='xlsxwriter')

for name in names:
    dfname = all_cf[['TS',name]].copy()
    for i in range(20,56):
        dfname['20{}'.format(i)]=all_cf[name]
        dfname.rename({'2055':'2055:=','TS':' '},axis=1, inplace=True)
        #dfname.drop([name], axis=1).to_excel('{}.xlsx'.format(name))
        dfname.drop([name], axis=1).to_excel(writer, sheet_name='{}'.format(name), index=False)

writer.save()

# Load curve and specified annual demand

In [29]:
# Year Split = (total number of hours in each time slice)/ total number of hours in a year

# Specified demand profile = (sum.hours in each time slice * sum.of demanded energy in each time slice)/Total annual energy



In [46]:
folder_path = os.path.join('Data', 'electricity_data')

load_curve = os.path.join(folder_path, 'Albania_load_curve.xlsx')

load = pd.read_excel(load_curve)

In [47]:
load #Note that the unit of load in the data file is MW. 

,Date,Load
0,2019-01-01 01:00:00.000,890.214891
1,2019-01-01 02:00:00.000,758.928970
2,2019-01-01 03:00:00.000,631.847777
3,2019-01-01 04:00:00.000,561.831515
4,2019-01-01 05:00:00.000,530.478296
...,...,...
8755,2019-12-31 19:59:59.998,1380.886928
8756,2019-12-31 20:59:59.998,1259.884561
8757,2019-12-31 21:59:59.998,1120.765704
8758,2019-12-31 22:59:59.998,1048.766506


In [48]:
load['year'] = load['Date'].dt.year 
load['month'] = load['Date'].dt.month
load['week'] = load['Date'].dt.week
load['day'] = load['Date'].dt.day 
load['hours'] = (1+load['Date'].dt.hour)/(1+load['Date'].dt.hour) 

In [49]:
load.rename(columns={'Load':'Power(MW)'}, inplace=True)
load.head()

,Date,Power(MW),year,month,week,day,hours
0,2019-01-01 01:00:00,890.214891,2019,1,1,1,1.0
1,2019-01-01 02:00:00,758.928970,2019,1,1,1,1.0
2,2019-01-01 03:00:00,631.847777,2019,1,1,1,1.0
3,2019-01-01 04:00:00,561.831515,2019,1,1,1,1.0
4,2019-01-01 05:00:00,530.478296,2019,1,1,1,1.0


In [50]:
#avg = pd.pivot_table(load, values='Power(MW)', index='week', columns=['filename'], aggfunc=np.mean, fill_value=0)
demand_profile = pd.pivot_table(load, values=['Power(MW)','hours'], index='week', aggfunc=np.sum, fill_value=0)
demand_profile.tail()

,Power(MW),hours
week,,
48,124589.795276,168
49,144569.481122,168
50,148703.144058,168
51,149184.447889,168
52,158825.687832,168


In [51]:
demand_profile2 = load.groupby('week')['Power(MW)','hours'].sum()
demand_profile2['year'] = load.groupby('week')['year'].mean()
demand_profile2['yearsplit'] = 8760/52/8760
demand_profile2['TSdemand'] = (demand_profile2['Power(MW)']*demand_profile2['hours'])
demand_profile2['specifieddemandprofile'] = demand_profile2['TSdemand']/(demand_profile2['TSdemand'].sum())
#demand_profile2['year'] = load.groupby('week')['year'].mean()
#demand_profile2[''] = load.groupby('week')['hour'].sum()
demand_profile2

<ipython-input-51-95b9acc40c57>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  demand_profile2 = load.groupby('week')['Power(MW)','hours'].sum()


,Power(MW),hours,year,yearsplit,TSdemand,specifieddemandprofile
week,,,,,,
1,193702.972924,191.0,2019,0.019231,3.699727e+07,0.030584
2,181364.828486,168.0,2019,0.019231,3.046929e+07,0.025188
3,174015.542461,168.0,2019,0.019231,2.923461e+07,0.024167
4,161636.726738,168.0,2019,0.019231,2.715497e+07,0.022448
5,153252.531899,168.0,2019,0.019231,2.574643e+07,0.021284
6,143452.283307,168.0,2019,0.019231,2.409998e+07,0.019923
7,154699.595571,168.0,2019,0.019231,2.598953e+07,0.021485
8,149052.463036,168.0,2019,0.019231,2.504081e+07,0.020700
9,153689.237657,168.0,2019,0.019231,2.581979e+07,0.021344


In [52]:
demand_profile2['yearsplit'].sum()

1.0

In [53]:
demand_profile2.reset_index(inplace=True)
demand_profile2.columns

Index(['week', 'Power(MW)', 'hours', 'year', 'yearsplit', 'TSdemand',
       'specifieddemandprofile'],
      dtype='object')

In [58]:
df_sdp = demand_profile2[['week','specifieddemandprofile']].copy()
df_ys = demand_profile2[['week','yearsplit']].copy()

In [59]:
df_sdp

,week,specifieddemandprofile
0,1,0.030584
1,2,0.025188
2,3,0.024167
3,4,0.022448
4,5,0.021284
5,6,0.019923
6,7,0.021485
7,8,0.020700
8,9,0.021344
9,10,0.019437


In [62]:
output_folder = os.path.join('Processed_data', 'Demand_Profile')
os.makedirs(output_folder, exist_ok = True)

file1_path=os.path.join(output_folder, 'Specified_demand_profile.xlsx')
file2_path=os.path.join(output_folder, 'year_split.xlsx')

df_sdp = demand_profile2[['week','specifieddemandprofile']].copy()
df_ys = demand_profile2[['week','yearsplit']].copy()

for i in range(20,56):
    df_sdp['20{}'.format(i)]=df_sdp['specifieddemandprofile'].round(3)
    #df_sdp.drop('specifieddemandprofile',axis=1, inplace=True)
    df_sdp.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    
    df_ys['20{}'.format(i)]=df_ys['yearsplit'].round(3)
    #df_ys.drop('yearsplit',axis=1, inplace=True)
    df_ys.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    

df_sdp.drop('specifieddemandprofile',axis=1, inplace=True)
df_ys.drop('yearsplit',axis=1, inplace=True)
writer1 = pd.ExcelWriter(file1_path, engine='xlsxwriter')
writer2 = pd.ExcelWriter(file2_path, engine='xlsxwriter')
df_sdp.to_excel(writer1, index=False)
df_ys.to_excel(writer2, index=False)

writer1.save()
writer2.save()